In [1]:
!pip install pdfminer pypdf2

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 4.2 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 11.7 MB/s eta 0:00:01
     |████████████████████████████████| 13.7 MB 19.1 MB/s eta 0:00:01
Using legacy setup.py install for pdfminer, since package 'wheel' is not installed.
Using legacy setup.py install for pypdf2, since package 'wheel' is not installed.
    Running setup.py install for pdfminer ... done
    Running setup.py install for pypdf2 ... done


In [225]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable


In [184]:
!exa -l data

drwxr--r-- - marcin 24 Jul  1:51 Alabama
drwxr--r-- - marcin 27 Jul 17:57 Arkansas
drwxr--r-- - marcin  4 May 23:02 Berkeley
drwxr--r-- - marcin  3 Dec 17:20 Bowdoin
drwxr--r-- - marcin  3 Dec 15:32 Clemson
drwxr--r-- - marcin  3 Dec 17:09 Colby
drwxr--r-- - marcin 27 May 18:07 Colorado - Boulder
drwxr--r-- - marcin 29 Jul 18:20 Colorado State
drwxr--r-- - marcin  3 Dec 16:11 CSU Fresno
drwxr--r-- - marcin  3 Dec 18:00 Duke
drwxr--r-- - marcin 13 Jul 19:05 George Mason
drwxr--r-- - marcin  3 Dec 17:43 Gonzaga
drwxr--r-- - marcin  3 Dec 17:25 Maine - Augusta
drwxr--r-- - marcin  3 Dec 17:47 Marshall
drwxr--r-- - marcin 27 May 18:22 Ohio State
drwxr--r-- - marcin 29 Jul 19:35 Old Dominion
drwxr--r-- - marcin  3 Dec 16:31 Oregon
drwxr--r-- - marcin  3 Dec 20:01 Swarthmore
drwxr--r-- - marcin  3 Dec 17:01 UCLA
drwxr--r-- - marcin 27 May 18:23 West Virginia
drwxr--r-- - marcin  3 Dec 20:02 William Carey


In [185]:
!exa data -T

data
├── Alabama
│  ├── 2010-2012.pdf
│  ├── 2012-2013.pdf
│  ├── 2013-2014.pdf
│  ├── 2014-2015.pdf
│  ├── 2015-2016.pdf
│  ├── 2016-17.pdf
│  ├── 2016-2017.pdf
│  ├── 2017-2018.pdf
│  ├── 2018-2019.pdf
│  └── 2019-2020.pdf
├── Arkansas
│  ├── 2001-2002.pdf
│  ├── 2002-2003.pdf
│  ├── 2003-2004.pdf
│  ├── 2004-2005.pdf
│  ├── 2006-2007.pdf
│  ├── 2007-2008.pdf
│  ├── 2008-2009.pdf
│  ├── 2009-2010.pdf
│  ├── 2010-2011.pdf
│  ├── 2011-2012.pdf
│  ├── 2012-2013.pdf
│  └── 2019-2020.pdf
├── Berkeley
│  ├── 1988_1989_courses.pdf
│  ├── 1989_1990_courses.pdf
│  ├── 1991_1992_courses.pdf
│  ├── 1993_1994_courses.pdf
│  ├── 1997_1999_courses.pdf
│  ├── 2001_2003_courses.pdf
│  ├── 2003_2005_courses.pdf
│  ├── 2005_2007_courses.pdf
│  ├── 2007_2009_courses.pdf
│  ├── 2009_2011_courses.pdf
│  └── 2011_2013_courses.pdf
├── Bowdoin
│  ├── Bowdoin_2009-2010.pdf
│  ├── Bowdoin_2010-2011.pdf
│  ├── Bowdoin_2011-2012.pdf
│  ├── Bowdoin_2012-2013.pdf
│  ├── Bowdoin_2013-2014.pdf
│  ├── Bowdoin_2014-2

In [226]:
import io
import re
import os

import pandas as pd
from tqdm.notebook import trange, tqdm

In [216]:
from PyPDF2 import PdfFileReader

def pdf2txt(path):
    txt = ''
    pdf = PdfFileReader(str(path))
    for page in pdf.pages:
        txt += page.extractText()
    return txt

In [6]:
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage

def pdf_to_text(path):
    with open(path, 'rb') as fp:
        rsrcmgr = PDFResourceManager()
        outfp = io.StringIO()
        laparams = LAParams()
        device = TextConverter(rsrcmgr, outfp, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.get_pages(fp):
            interpreter.process_page(page)
    text = outfp.getvalue()
    return text

In [ ]:
parse_only = ['Arkansas','Bowdoin','Ohio State']
parse_options = {
    "Alabama": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "Arkansas": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "Berkeley": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "Bowdoin": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "Clemson": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "Colby": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "Colorado - Boulder": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "Colorado State": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "CSU Fresno": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "Duke": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "George Mason": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "Gonzaga": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "Maine - Augusta": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "Marshall": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "Ohio State": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "Old Dominion": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "Oregon": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "Swarthmore": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "UCLA": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "West Virginia": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
    "William Carey": {
        "fn": pdf_to_text, 
        "regex": (
            r""
        )
    },
}

In [33]:
done = False
for directory,_,filenames in os.walk('data'):
    if directory.lstrip('data/') in parse_only:
        for filename in filenames:
            data = pdf_to_text(directory+'/'+filename)
            done = True
            break
    if done: break

# Arkansas

In [43]:
arkansas_sample = pdf_to_text('data/Arkansas/2019-2020.pdf')

In [122]:
re.findall(r"(\n\n?[A-Z]{4}\s[0-9]{3,4}\.[\(\w\s.\n,\.:-]*)",arkansas_sample)

['\nENGL 0002.\n\n',
 '\nMATH 1203.\n\nStudents will be required to register for these courses during their first\nterm at the university and, if necessary, in subsequent terms until passing\ngrades have been earned in all required courses. Students who need\nfurther information or clarification are encouraged to discuss their course\nplacement with their academic advisor or dean.\n\nCourses That Do Not Count Toward a\nDegree\nThe following courses do not count toward degree credit in any college\nor school: ENGL 0002, ENGL 0013, MATH 0003, MATH 0001L,\nMATH 0002L and MATH 0131L.\n\nThe following courses do not count toward any degree in the College of\nEngineering: MATH 1203, MATH 1213, MATH 1284C, and ENGL 2003.\n\nSpeech Communication Exemption\nExamination\nStudents who have had speech in high school and',
 '\nCOMM 1313. Both the written and oral (a five-minute impromptu speech',
 '\nPLSC 2003.\n\n98         University Core\n\nand judgments derived from experience in the core area 

# Bowdowin

In [110]:
# pdf2txt doesnt work
bowdowin_sample = pdf_to_text("data/Bowdoin/Bowdoin_2019-2020.pdf")
bs = bowdowin_sample

In [346]:
# find = "AFRS 1012"
# bs[bs.index(find)-10:bs.index(find)+2000]
bowdoin_dfs = []
for file in tqdm(os.listdir('data/Bowdoin')):
    text = pdf_to_text(f"data/Bowdoin/{file}")
    if '202' in file:
        data = [{'id':r[1],'title':r[3],'sub':r[4],'desc':r[6],'num':r[5],'catalog':file} 
            for r in re.findall(
                r"("
                    r"\n\n([A-Z]{4}(\s|\\xa0)[0-9]{4})"  # course id
                    r"(.*?)\n"                           # course title
                    r"(.*?)"                             # course subtitle
                    r"\s([0-9]{1,3})\."                  # course enrolment limit
                    r"((?:(?![\n]{1}[A-Z]{4}).)*)"       # course description 
                                                          # UNTIL next course id 
                                                          # but not including it
                r")",
                text,
                re.DOTALL)
        ]
    else: #pre 2020
        data = [{'id':r[1],'title':r[3],'sub':None,'desc':r[4],'num':None,'catalog':file} 
                for r in re.findall(
                r"("
                    r"\n([0-9]{2,3}[a-z]{0,1})(\.\s|\s-)"  # course id
                    r"(.*?)\n"                           # course title
                    r"((?:(?!\n([0-9]{2,3}[a-z]{0,1})(\.\s|\s-)).)*)"  # UNTIL next course id but not including
                r")",
                text,
                re.DOTALL)
        ]
        
    bowdoin_dfs.append(pd.DataFrame(data))

In [347]:
bowdoin_dfs

[Empty DataFrame
 Columns: []
 Index: [],
        id                                              title   sub  \
 0     10b  Racism. Fall 2010. Spring 2011. Roy Partridge....  None   
 1     13c  From Montezuma to Bin Laden: Globalization and...  None   
 2     16c  Fictions of Freedom. Fall 2010. Tess Chakkalak...  None   
 3    101c  Introduction to Africana Studies. Every fall. ...  None   
 4    107c   ESD. Introduction to the Black Novel in the U...  None   
 ..    ...                                                ...   ...   
 869  321c  Comedy in Performance. Every third year. Sprin...  None   
 870  322c  Ensemble: Collaborative Creation. Spring 2011....  None   
 871  323c  Acting Shakespeare. Spring 2012. Abigail Killeen.  None   
 872  340c  Live Performance and Digital Media. Fall 2010....  None   
 873  370c   Advanced Directing. Spring 2013. The Department.  None   
 
                                                   desc   num  \
 0    12c. Affirmative Action and Unite

In [311]:
# # 2020 formula
# re.findall(
#     r"("
#         r"\n\n([A-Z]{4}(\s|\\xa0)[0-9]{4})"  # course id
#         r"(.*?)\n"                           # course title
#         r"(.*?)"                             # course subtitle
#         r"\s([0-9]{1,3})\."                  # course enrolment limit
# #         r"((.*?))"                             # course description
#         r"((?:(?![\n]{1}[A-Z]{4}).)*)"  # UNTIL next course id but not including
#     r")",
#                 bs,
#                 re.DOTALL)
        

In [345]:
# # pre 2020 formula
# bowdowin_sample = pdf_to_text("data/Bowdoin/Bowdoin_2011-2012.pdf")
# bowdowin_sample = pdf_to_text("data/Bowdoin/Bowdoin_2018-2019.pdf")


re.findall(
    r"("
        r"\n([0-9]{2,3}[a-z]{0,1})(\.\s|\s-)"  # course id
        r"(.*?)\n"                           # course title
        r"((?:(?!\n([0-9]{2,3}[a-z]{0,1})(\.\s|\s-)).)*)"  # UNTIL next course id but not including
    r")",
                bowdowin_sample,
                re.DOTALL)
bowdowin_sample

'B\nO\nW\nD\nO\n\nI\n\nN\n\n \n\nC\nO\nL\nL\nE\nG\nE\n\n \n\nA\nC\nA\nD\nE\nM\n\nI\n\nC\n\n \n\nH\nA\nN\nD\nB\nO\nO\nK\n\n \n\n2\n0\n1\n8\n–\n2\n0\n1\n9\n\nA C A D E M I C   H A N D B O O K : \nPOLICIES AND PROCEDURES \n\n201 8–201 9\n\nAugust 2018\nBrunswick, Maine\nbowdoin.edu/academic-handbook\n\nOffice of the Dean for Academic Affairs\nBowdoin College\n5800 College Station\nBrunswick, ME \n04011-8449\n\n\x0cB\nO\nW\nD\nO\n\nI\n\nN\n\n \n\nC\nO\nL\nL\nE\nG\nE\n\n \n\nA\nC\nA\nD\nE\nM\n\nI\n\nC\n\n \n\nH\nA\nN\nD\nB\nO\nO\nK\n\n \n\n2\n0\n1\n8\n–\n2\n0\n1\n9\n\nOffice of the Dean for Academic Affairs\nBowdoin College\n5800 College Station\nBrunswick, ME \n04011-8449\n\n\x0cACADEMIC HANDBOOK \n\nPOLICIES AND PROCEDURES \n\n2018–2019\n\nbowdoin.edu/academic-handbook\n\nAugust 2018\n\n\x0cBowdoin College complies with applicable provisions of federal and state laws that prohibit \nunlawful discrimination in employment, admission, or access to its educational or extracurricular \nprogram

# Ohio State

In [133]:
# ohio_sample = pdf_to_text("data/Ohio State/Course Bulletin 2019-2020.pdf")
# ohs = ohio_sample
# [r 
#  for r 
#  in re.findall(
#      r"("
#         r"\n\n("                           # course ID
#             r"([0-9]{4}\.[0-9A-Z]{3})"     # e.g. 2367.07S
#             r"|([0-9]{4}\.[0-9]{2})"      # e.g. 2367.04
#             r"|([0-9]{4}[H])"              # e.g. 4780H
#             r"|([0-9]{4})"                 # e.g. 3080
#         r")"
#        r"(\n\n[A-Za-z\s]*)"                # course title
# #      r"\n\n[U]{1}\n\n[0-9]{1}"
#      r".*?\n"
#      r")"
#      ,ohs, re.DOTALL) # \s[0-9]{3,4}[\(\w\s.\n,\.:-]*)
# #  if '\xa0\xa0(' not in r
# ]

In [229]:
# df_ohio = pd.DataFrame(columns=['id','title','desc','num','catalog'])
ohio_dfs = []
for file in tqdm(os.listdir('data/Ohio State')):
    # better to use pdf2txt, otherwise the number to the left of letter U
    # sometimes flotes away from the course it is assigned to
    ohs2 = pdf2txt(f"data/Ohio State/{file}")

    data = [{'id':r[1],'title':r[6],'desc':r[7],'num':r[8],'catalog':file} 
     for r 
     in re.findall(
         r"("
            r"\n("                           # course ID
                r"([0-9]{4}\.[0-9A-Z]{3})"     # e.g. 2367.07S
                r"|([0-9]{4}\.[0-9]{2})"       # e.g. 2367.04
                r"|([0-9]{4}[H])"              # e.g. 4780H
                r"|([0-9]{4})"                 # e.g. 3080
            r")"
           r"\n([A-Za-z\s]*)"                # course title
           r"\nU\n"                          # letter U
           r"(.*?)"                          # course desrciption
           r"\n([0-9]{1,2})"                 # number to the right of letter U
         r")",
         ohs2, 
         re.DOTALL) 
    ]
#     df_ohio.append(pd.DataFrame(data))
    ohio_dfs.append(pd.DataFrame(data))
    

In [238]:
df_ohio = pd.concat(ohio_dfs)
.to_csv('data/ohio.csv')